In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import os
path=os.getcwd()+ '\data\mnist_train.csv'   #get current working directory for data size values
raw_data=pd.read_csv(path)
raw_data.head()


,2,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.712,0.713,0.714,0.715,0.716,0.717,0.718,0.719,0.720,0.721
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
raw_data.shape   #4 classes, for every class-785 thetas--- size of theta matrix=features in each class*no of classes=785*4

(449, 785)

In [4]:
cols=raw_data.shape[1]
print("columns= {}".format(cols))

columns= 785


In [5]:
y=np.array(raw_data.iloc[:,:1])
x=np.array(raw_data.iloc[:,1:cols])
x.shape,y.shape

((449, 784), (449, 1))

In [6]:
b=np.unique(y)
print(b)     #b is all unique classes in the output

[0 1 2 3]


In [7]:
classes=b.size
print(classes)

4


In [8]:
rows=x.shape[0]   

In [9]:
x=np.insert(x,0,values=np.ones(rows),axis=1)     #inserting ones into x so now x becomes a 450*(784+1)matrix
x.shape

(449, 785)

In [10]:
params=x.shape[1]    #finding the number of features

In [11]:
all_theta=np.zeros((4,params))    #4 denotes number of classes
#all_theta=np.matrix(all_theta)
all_theta.shape

(4, 785)

In [12]:
def sigmoid(z):
    return 1/(1+np.exp(-z))
print(sigmoid(0))
print (sigmoid(10))
print(sigmoid(100))
print(sigmoid(1000))
#no matter what value we pass, sigmoid is always between 0 and 1

0.5
0.9999546021312976
1.0
1.0


In [13]:
def computeCost(all_theta,x,y):
    all_theta=np.matrix(all_theta)
    x=np.matrix(x)
    y=np.matrix(y)
    first=np.multiply(-y,np.log(sigmoid(x*all_theta.T)))
    second=np.multiply((1-y),np.log(1-sigmoid(x*all_theta.T)))
    cost=np.sum(first-second)
    cost=cost/(2*len(x))
    return cost
cost=computeCost(all_theta,x,y)
print("cost={}".format(cost))

cost=1.3862943611198906


In [14]:
learning_rate=1
def gradient(theta,x,y):
    theta=np.matrix(theta)
    x=np.matrix(x)
    y=np.matrix(y)
    error=sigmoid(x*theta.T)-y
    grad=((x.T*error)/len(x))
    grad=grad*learning_rate
    return grad


In [15]:
from scipy.optimize import minimize
def one_vs_all(x,y,classes,new_theta):
    params=x.shape[1]
    rows=x.shape[0]
    for i in range(0,classes):
        theta=np.zeros(params)
        theta=new_theta[i,:]
        y_i=np.array([1 if label == i else 0 for label in y])
        y_i=np.reshape(y_i,(rows,1))
        fmin=minimize(fun=computeCost,x0=theta,args=(x,y_i),method='TNC',jac=gradient)
        all_theta[i,:]=fmin.x
        return all_theta
    

In [16]:
new_theta=one_vs_all(x,y,classes,all_theta)

C:\Users\Lenovo y540 bnin\Desktop\ML\Lema Labs ML Workshop_x64\Lema Labs ML Workshop x64\python-3.7.2.amd64\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  
C:\Users\Lenovo y540 bnin\Desktop\ML\Lema Labs ML Workshop_x64\Lema Labs ML Workshop x64\python-3.7.2.amd64\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in multiply
  


In [17]:
computeCost(new_theta,x,y)

17.22546016765499

In [18]:
def predict_all(x,theta):
    #convert to matrices
    x=np.matrix(x)
    theta=np.matrix(theta)
    #compute the classprobab for each class on each training instance
    h=sigmoid(x*theta.T)
    #create array of the index with max probab
    h_argmax=np.argmax(h,axis=1)
    #because our araay was not zero-indexed as we need not add one for the true label prediction
    return h_argmax,h

In [19]:
y_pred,h=predict_all(x, new_theta)
print(h)

[[8.62195284e-09 5.00000000e-01 5.00000000e-01 5.00000000e-01]
 [8.47697506e-10 5.00000000e-01 5.00000000e-01 5.00000000e-01]
 [3.60065508e-07 5.00000000e-01 5.00000000e-01 5.00000000e-01]
 ...
 [3.74910381e-09 5.00000000e-01 5.00000000e-01 5.00000000e-01]
 [1.16100078e-05 5.00000000e-01 5.00000000e-01 5.00000000e-01]
 [5.01302915e-09 5.00000000e-01 5.00000000e-01 5.00000000e-01]]


In [20]:
correct=[1 if a==b else 0 for (a,b) in zip(y_pred,y)]
print(correct)
accuracy=(sum(map(int,correct))/float(len(correct)))
print("Training data={}%".format(accuracy*100))

[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 

In [21]:
from PIL import Image
from PIL import ImageOps
from IPython.display import display
Im= Image.open('/pic.png')
display(Im)

FileNotFoundError: [Errno 2] No such file or directory: '/pic.png'

In [22]:
# funtion to convert rgb to grab
Im=Im.convert('L') #convert to gray scale
Im=ImageOps.invert(Im) #invert the image
display(Im)

NameError: name 'Im' is not defined

In [23]:
#converting the image into a single uni-dimensional array
Im=np.array(Im)
Im=np.ravel(Im)
Im.shape

NameError: name 'Im' is not defined

In [24]:
Im=Im.reshapee(1,-1)

NameError: name 'Im' is not defined

In [25]:
input_x=np.insert(Im,0,1,axis=1)

NameError: name 'Im' is not defined

In [27]:
#predicting the new input using the theta that we found using oneVsAll
pred,h=predict_all(input_x,new_theta)
print("confidence matrix={}".format(h))
print("Predicted class is :{}".format(pred.item()))
print("Confidence={}%".format(((h[0,pred])*100).item()))

NameError: name 'input_x' is not defined